In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()
base_path = Path.home() / Path(os.environ.get("DATA_PATH"))
print(base_path)

/home/aayush/rsys_data


In [2]:
df = pd.read_csv(
   base_path/ Path("behaviors.tsv"), 
    sep="\t",
    names=["impressionId","userId","timestamp","click_history","impressions"])

print(f"The dataset originally consist of {len(df)} number of interactions.")


df.head()

The dataset originally consist of 2232748 number of interactions.


,impressionId,userId,timestamp,click_history,impressions
0,1,U87243,11/10/2019 11:30:54 AM,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...
1,2,U598644,11/12/2019 1:45:29 PM,N56056 N8726 N70353 N67998 N83823 N111108 N107...,N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...
2,3,U532401,11/13/2019 11:23:03 AM,N128643 N87446 N122948 N9375 N82348 N129412 N5...,N103852-0 N53474-0 N127836-0 N47925-1
3,4,U593596,11/12/2019 12:24:09 PM,N31043 N39592 N4104 N8223 N114581 N92747 N1207...,N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...
4,5,U239687,11/14/2019 8:03:01 PM,N65250 N122359 N71723 N53796 N41663 N41484 N11...,N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...


In [3]:
df['impressions'].dtype

dtype('O')

In [4]:
def process_impression(impression_list):
    list_of_strings = impression_list.split()
    click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '1']
    non_click = [x.split('-')[0] for x in list_of_strings]
    return click,non_click


df['click'], df['presented_slate'] = zip(*df['impressions'].map(process_impression))
df

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,1,U87243,11/10/2019 11:30:54 AM,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,"[N94157, N78699, N71090, N31174]","[N78206, N26368, N7578, N58592, N19858, N58258..."
1,2,U598644,11/12/2019 1:45:29 PM,N56056 N8726 N70353 N67998 N83823 N111108 N107...,N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,"[N25587, N36266]","[N47996, N82719, N117066, N8491, N123784, N214..."
2,3,U532401,11/13/2019 11:23:03 AM,N128643 N87446 N122948 N9375 N82348 N129412 N5...,N103852-0 N53474-0 N127836-0 N47925-1,[N47925],"[N103852, N53474, N127836, N47925]"
3,4,U593596,11/12/2019 12:24:09 PM,N31043 N39592 N4104 N8223 N114581 N92747 N1207...,N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...,[N114935],"[N38902, N76434, N71593, N100073, N108736, N30..."
4,5,U239687,11/14/2019 8:03:01 PM,N65250 N122359 N71723 N53796 N41663 N41484 N11...,N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...,[N86258],"[N76209, N48841, N67937, N62235, N6307, N34254..."
...,...,...,...,...,...,...,...
2232743,2232744,U316192,11/13/2019 6:50:02 PM,N122359 N37069 N95876 N28787 N73408 N11266 N61321,N113723-0 N123683-1 N5287-0 N76677-0 N53474-0,[N123683],"[N113723, N123683, N5287, N76677, N53474]"
2232744,2232745,U451238,11/12/2019 8:54:06 AM,N12575 N93816 N71643 N87236 N87236,N18861-0 N20990-0 N43085-0 N7937-1,[N7937],"[N18861, N20990, N43085, N7937]"
2232745,2232746,U151246,11/13/2019 12:42:51 PM,N27587 N49668,N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,"[N39887, N110709, N24001, N127572, N10285]","[N39887, N22811, N110709, N1923, N24001, N7667..."
2232746,2232747,U330725,11/12/2019 1:22:57 PM,N121944 N91510 N42280 N60061 N63032 N125223 N4...,N18947-0 N88808-1 N10012-0 N38902-0 N33078-0 N...,[N88808],"[N18947, N88808, N10012, N38902, N33078, N1003..."


In [5]:
news = pd.read_csv(
    base_path/ Path("news.tsv"), 
    sep="\t",
    names=["itemId","category","subcategory","title","abstract","url","title_entities","abstract_entities"])
print(f"The article data consist in total of {len(news)} number of articles.")
news

The article data consist in total of 101527 number of articles.


,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
...,...,...,...,...,...,...,...,...
101522,N115249,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]
101523,N64337,finance,finance-real-estate,Mansion Monday: Contemporary Des Moines home i...,Among the perks of this unique Des Moines home...,https://assets.msn.com/labs/mind/BBWzQq8.html,"[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W...","[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W..."
101524,N100102,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[]
101525,N74617,autos,autossports,Best Sports Car Deals for October,NaN,https://assets.msn.com/labs/mind/BBy5rVe.html,"[{""Label"": ""Peugeot RCZ"", ""Type"": ""V"", ""Wikida...",[]


In [6]:
valid_article_ids = set(news['itemId'])
print(len(news['itemId']))
def filter_click_list(click_list):
    return [item for item in click_list if item in valid_article_ids]
def filter_click_history_list(click_list):
    if pd.notna(click_list['click_history']):
     return [item for item in click_list['click_history'].split() if item in valid_article_ids]
    else:
     return []
behaviors_df=df

behaviors_df['click'] = behaviors_df['click'].apply(filter_click_list)
behaviors_df['presented_slate'] = behaviors_df['presented_slate'].apply(filter_click_list)
behaviors_df['click_history'] = behaviors_df.apply(filter_click_history_list,axis=1)
# behaviors_df[behaviors_df['click'].apply(lambda x: any(item in valid_article_ids for item in x))]
# filtered_behaviors_df = filtered_behaviors_df[filtered_behaviors_df['presented_slate'].apply(lambda x: any(item in valid_article_ids for item in x))]
filtered_behaviors_df = behaviors_df[behaviors_df['click'].apply(lambda x: len(x) > 0)]
filtered_behaviors_df


101527


,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,"[N94157, N78699, N71090, N31174]","[N78206, N26368, N7578, N58592, N19858, N58258..."
1,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,"[N25587, N36266]","[N47996, N82719, N117066, N8491, N123784, N214..."
2,3,U532401,11/13/2019 11:23:03 AM,"[N128643, N87446, N122948, N9375, N82348, N129...",N103852-0 N53474-0 N127836-0 N47925-1,[N47925],"[N103852, N53474, N127836, N47925]"
3,4,U593596,11/12/2019 12:24:09 PM,"[N31043, N39592, N4104, N8223, N114581, N92747...",N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...,[N114935],"[N38902, N76434, N71593, N100073, N108736, N30..."
4,5,U239687,11/14/2019 8:03:01 PM,"[N65250, N122359, N71723, N53796, N41663, N414...",N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...,[N86258],"[N76209, N48841, N67937, N62235, N6307, N34254..."
...,...,...,...,...,...,...,...
2232743,2232744,U316192,11/13/2019 6:50:02 PM,"[N122359, N37069, N95876, N28787, N73408, N112...",N113723-0 N123683-1 N5287-0 N76677-0 N53474-0,[N123683],"[N113723, N123683, N5287, N76677, N53474]"
2232744,2232745,U451238,11/12/2019 8:54:06 AM,"[N12575, N93816, N71643, N87236, N87236]",N18861-0 N20990-0 N43085-0 N7937-1,[N7937],"[N18861, N20990, N43085, N7937]"
2232745,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,"[N39887, N110709, N24001, N127572, N10285]","[N39887, N22811, N110709, N1923, N24001, N7667..."
2232746,2232747,U330725,11/12/2019 1:22:57 PM,"[N121944, N91510, N42280, N60061, N63032, N125...",N18947-0 N88808-1 N10012-0 N38902-0 N33078-0 N...,[N88808],"[N18947, N88808, N10012, N38902, N33078, N1003..."


In [7]:
news[news['itemId'] == 'N82719']

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
87005,N82719,news,newsus,Widow Sues Truth Aquatics Owner In Conception ...,The widow of a man who died in when a dive boa...,https://assets.msn.com/labs/mind/BBWBDEF.html,[],[]


In [8]:
# filtered_df = df[
#     (df['click'].apply(lambda x: len(x) >= 5)) &
#     (df['presented_slate'].apply(lambda x: 5 <= len(x) <= 10))
# ]

In [9]:
# filtered_df = df[
#     (df['click'].apply(lambda x: len(x) >= 5)) &
#     (df['presented_slate'].apply(lambda x: 5 <= len(x) <= 10))
# ]
# grouped_filtered_df = filtered_df.sort_values(by='timestamp').groupby('userId').agg({
#     'impressions': lambda x: list(x),
#     'timestamp': lambda x: list(x),
#     'click_history': lambda x: list(x),
#     'click': lambda x: list(x),
#     'presented_slate': lambda x: list(x),   
# }).reset_index()
# grouped_filtered_df

In [10]:
# ls=grouped_filtered_df[grouped_filtered_df['userId'] == 'U10'][['presented_slate'][0]].tolist()
# ls

In [11]:
# filtered_df_3 = df[
#     (df['click'].apply(lambda x: len(x) >= 3)) &
#     (df['presented_slate'].apply(lambda x: 5 <= len(x) <= 10))
# ]

In [12]:
filtered_df_2 = filtered_behaviors_df[
    (df['click'].apply(lambda x: len(x) != 1 or (len(x) == 0 and isinstance(x, list)))) &
    (df['presented_slate'].apply(lambda x: 5 <= len(x)))
].reset_index()

In [13]:
filtered_df_2

,index,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,"[N94157, N78699, N71090, N31174]","[N78206, N26368, N7578, N58592, N19858, N58258..."
1,1,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,"[N25587, N36266]","[N47996, N82719, N117066, N8491, N123784, N214..."
2,13,14,U95671,11/14/2019 8:28:43 PM,"[N64593, N82779, N33216, N9321, N128643, N6449...",N62736-0 N52388-0 N16161-0 N127585-0 N88596-0 ...,"[N51048, N39403]","[N62736, N52388, N16161, N127585, N88596, N433..."
3,18,19,U419371,11/12/2019 4:15:12 AM,"[N129790, N59633, N51088, N126521, N882, N4570...",N113644-0 N22659-0 N59301-0 N78239-0 N91047-0 ...,"[N120192, N87838, N95254, N53859]","[N113644, N22659, N59301, N78239, N91047, N416..."
4,24,25,U552512,11/12/2019 8:51:10 PM,"[N114076, N83597, N38169, N8402, N3773, N10604...",N5098-1 N82866-0 N58417-0 N74117-0 N33468-0 N2...,"[N5098, N75494, N47643, N80444, N44037, N18947...","[N5098, N82866, N58417, N74117, N33468, N26837..."
...,...,...,...,...,...,...,...,...
615591,2232732,2232733,U510519,11/12/2019 6:24:48 AM,"[N70946, N57593, N9276, N5163, N63379, N40628,...",N43085-0 N100073-1 N20990-0 N2750-0 N27097-0 N...,"[N100073, N30899, N83412]","[N43085, N100073, N20990, N2750, N27097, N1217..."
615592,2232735,2232736,U182931,11/14/2019 6:03:16 AM,"[N87446, N104938, N49247, N112324, N103181, N8...",N18405-0 N76665-0 N25751-0 N5747-0 N85664-0 N4...,"[N20250, N58641]","[N18405, N76665, N25751, N5747, N85664, N45522..."
615593,2232737,2232738,U424578,11/13/2019 10:44:02 AM,"[N65123, N63858, N2661, N40628, N98238, N98238...",N88472-0 N106659-0 N51686-0 N10285-0 N127836-0...,"[N100417, N72609, N93856]","[N88472, N106659, N51686, N10285, N127836, N48..."
615594,2232740,2232741,U189711,11/14/2019 8:17:31 PM,"[N26834, N86208, N83597, N90359, N91280, N5531...",N72848-0 N104990-0 N3623-0 N51307-0 N83707-0 N...,"[N91238, N86258, N102846]","[N72848, N104990, N3623, N51307, N83707, N1186..."


Interaction contains impression of size between 5 and 10 while interaction_all between 5 and greater.

In [14]:
filtered_df_2_1=filtered_df_2.explode('click', ignore_index=True)
filtered_df_2_1=filtered_df_2_1.drop(columns=['index'])
filtered_df_2_1
# feather_path=base_path/ Path("MINDlarge_train/interaction_all.feather")
# filtered_df_2_1.to_feather(feather_path)

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N94157,"[N78206, N26368, N7578, N58592, N19858, N58258..."
1,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N78699,"[N78206, N26368, N7578, N58592, N19858, N58258..."
2,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N71090,"[N78206, N26368, N7578, N58592, N19858, N58258..."
3,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N31174,"[N78206, N26368, N7578, N58592, N19858, N58258..."
4,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,N25587,"[N47996, N82719, N117066, N8491, N123784, N214..."
...,...,...,...,...,...,...,...
1763133,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N39887,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763134,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N110709,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763135,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N24001,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763136,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N127572,"[N39887, N22811, N110709, N1923, N24001, N7667..."


In [15]:
filtered_df_2_1

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N94157,"[N78206, N26368, N7578, N58592, N19858, N58258..."
1,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N78699,"[N78206, N26368, N7578, N58592, N19858, N58258..."
2,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N71090,"[N78206, N26368, N7578, N58592, N19858, N58258..."
3,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N31174,"[N78206, N26368, N7578, N58592, N19858, N58258..."
4,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,N25587,"[N47996, N82719, N117066, N8491, N123784, N214..."
...,...,...,...,...,...,...,...
1763133,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N39887,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763134,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N110709,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763135,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N24001,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763136,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N127572,"[N39887, N22811, N110709, N1923, N24001, N7667..."


In [16]:
# news['num_category'] = news['category'].factorize()[0]
# article_category_map = news.set_index('itemId')['num_category'].to_dict()

# def get_clicked_category_list(row):
#     # if pd.notna(row['click_history']):
#         # Use a list comprehension to create the list of categories
#     categories = [article_category_map.get(article_id, 0) for article_id in row['click']]
#     count_categories = [categories.count(i) for i in range(0, 18)]
#     # else:
#     #     # If 'click_history' is NaN, assign a list of zeros
#     #     count_categories = []
#     return count_categories
# def get_clicked_hist_category_list(row):
#     # if pd.notna(row['click_history']):
#         # Use a list comprehension to create the list of categories
#     categories = [article_category_map.get(article_id, 0) for article_id in row['click_history']]
#     count_categories = [categories.count(i) for i in range(0, 18)]
#     # else:
#     #     # If 'click_history' is NaN, assign a list of zeros
#     #     count_categories = []
#     return count_categories

# # Apply the function to create the new column
# filtered_df_2['clicked_category_list'] = filtered_df_2.apply(get_clicked_category_list, axis=1)
# filtered_df_2['clicked_hist_category_list'] = filtered_df_2.apply(get_clicked_hist_category_list, axis=1)
# filtered_df_2

In [17]:
# # correlation_matrix = filtered_df_2['clicked_category_list'].apply(pd.Series).corrwith(filtered_df_2['clicked_hist_category_list'].apply(pd.Series))
# correlation_matrix = filtered_df_2.apply(lambda x: pd.Series(x['clicked_category_list']).corr(pd.Series(x['clicked_hist_category_list'])), axis=1)
# # Print the correlation matrix
# print("Correlation Matrix:")
# print(correlation_matrix)

In [18]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# sns.heatmap([correlation_matrix], annot=True, cmap='coolwarm', fmt=".2f", linewidths=.2)
# plt.title('Correlation Matrix')
# plt.show()

In [19]:
news[news['itemId'] == 'N25587']

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
82510,N25587,news,newsus,Sen. Menendez: 'It's sickening' to hear Trump ...,"Sen. Bob Menedez, D-NJ, was inside the Supreme...",https://assets.msn.com/labs/mind/BBWEPyw.html,[],"[{""Label"": ""New Jersey Democratic State Commit..."


In [20]:
grouped_filtered_df_2 = filtered_df_2_1.sort_values(by='timestamp').groupby('userId').agg({
    'impressions': lambda x: list(x),
    'timestamp': lambda x: list(x),
    'click_history': lambda x: list(x),
    'click': lambda x: list(x),
    'presented_slate': lambda x: list(x),   
}).reset_index()
grouped_filtered_df_2

,userId,impressions,timestamp,click_history,click,presented_slate
0,U0,[N73137-0 N53751-0 N6865-0 N122343-0 N116601-0...,"[11/9/2019 6:44:01 AM, 11/9/2019 6:44:01 AM]","[[N39011, N112324, N78884, N111503, N63941, N6...","[N35189, N38813]","[[N73137, N53751, N6865, N122343, N116601, N36..."
1,U1,[N74049-0 N62203-0 N117342-0 N38152-0 N51219-0...,"[11/12/2019 2:24:19 PM, 11/12/2019 2:24:19 PM,...","[[N14639, N27258, N63237, N112729, N42180, N10...","[N17881, N36306, N3075, N26359, N92498, N31958...","[[N74049, N62203, N117342, N38152, N51219, N48..."
2,U100,[N58641-0 N76665-0 N84886-0 N20250-1 N125664-0...,"[11/14/2019 6:39:36 AM, 11/14/2019 6:39:36 AM]","[[N99587, N61339, N129790, N12721, N100405, N1...","[N41547, N20250]","[[N58641, N76665, N84886, N20250, N125664, N10..."
3,U10000,[N41315-0 N24701-0 N93856-0 N75618-0 N95525-0 ...,"[11/13/2019 11:45:01 AM, 11/13/2019 11:45:01 A...","[[N34918, N81659, N128643, N97343, N103301, N9...","[N100539, N103852, N82233, N67358, N10285, N13...","[[N41315, N24701, N93856, N75618, N95525, N110..."
4,U100001,[N70822-0 N14338-0 N11682-0 N31879-0 N123077-0...,"[11/11/2019 2:06:52 PM, 11/11/2019 2:06:52 PM]","[[N116367, N121227, N77370, N73122, N21475, N4...","[N98178, N47257]","[[N70822, N14338, N11682, N31879, N123077, N55..."
...,...,...,...,...,...,...
323475,U99992,[N40664-0 N75391-1 N160-0 N11789-0 N30899-0 N4...,"[11/11/2019 7:39:16 PM, 11/11/2019 7:39:16 PM,...","[[N42825, N30249, N129444, N107876, N27430, N7...","[N75391, N80790, N17907, N62235, N98412]","[[N40664, N75391, N160, N11789, N30899, N47110..."
323476,U99993,[N97369-0 N107542-0 N51166-0 N23920-0 N40996-0...,"[11/11/2019 9:18:43 AM, 11/11/2019 9:18:43 AM,...","[[N67397, N67770, N41129, N128503, N51724, N72...","[N50244, N129321, N61384, N122819, N17198]","[[N97369, N107542, N51166, N23920, N40996, N12..."
323477,U99994,[N14067-0 N88743-0 N24516-0 N127073-0 N18999-0...,"[11/11/2019 9:36:05 AM, 11/11/2019 9:36:05 AM]","[[N94577, N30729, N127177, N14611, N82747, N54...","[N126000, N93674]","[[N14067, N88743, N24516, N127073, N18999, N11..."
323478,U99996,[N110925-0 N29544-0 N105407-0 N25751-0 N48015-...,"[11/14/2019 8:11:11 AM, 11/14/2019 8:11:11 AM]","[[N113476, N54005, N129955, N14121, N73408, N1...","[N76665, N56873]","[[N110925, N29544, N105407, N25751, N48015, N1..."


In [21]:
max_length = grouped_filtered_df_2['click'].apply(len).max()
mean_length = grouped_filtered_df_2['click'].apply(len).mean()
min_length = grouped_filtered_df_2['click'].apply(len).min()
mode_length = grouped_filtered_df_2['click'].apply(len).mode().iloc[0]
median_length = grouped_filtered_df_2['click'].apply(len).median()

# Print the results
print("Max length:", max_length)
print("Mean length:", mean_length)
print("Min length:", min_length)
print("Mode length:", mode_length)
print("Median length:", median_length)

Max length: 165
Mean length: 5.450531717571411
Min length: 2
Mode length: 2
Median length: 3.0


In [22]:
grouped_filtered_df_2_t = filtered_df_2_1.sort_values(by='timestamp').groupby('timestamp').agg({
    'impressions': lambda x: list(x),
    'click_history': lambda x: list(x),
    'click': lambda x: list(x),
    'presented_slate': lambda x: list(x),   
}).reset_index()
grouped_filtered_df_2_t

,timestamp,impressions,click_history,click,presented_slate
0,11/10/2019 10:00:00 AM,[N64730-0 N54460-0 N26368-0 N58258-0 N7578-0 N...,"[[N72046, N47788, N66294, N91514, N48606, N715...","[N71090, N31174]","[[N64730, N54460, N26368, N58258, N7578, N1847..."
1,11/10/2019 10:00:02 AM,[N40282-0 N58258-0 N2591-0 N28809-0 N71090-1 N...,"[[N27258, N6408, N80170, N116695, N92191, N367...","[N31174, N71090]","[[N40282, N58258, N2591, N28809, N71090, N5446..."
2,11/10/2019 10:00:03 PM,[N32844-0 N107191-0 N19858-0 N91923-0 N109083-...,"[[N125760, N57532, N104554, N104081, N90185, N...","[N51158, N3664]","[[N32844, N107191, N19858, N91923, N109083, N5..."
3,11/10/2019 10:00:04 AM,[N28743-0 N36126-0 N121774-0 N58592-0 N44967-0...,"[[N97343, N61509, N17120, N73629, N15551, N693...","[N80175, N31174, N40282, N117508, N31174, N941...","[[N28743, N36126, N121774, N58592, N44967, N11..."
4,11/10/2019 10:00:05 AM,[N118944-0 N27427-0 N78699-0 N126722-0 N35936-...,"[[N2, N107944, N39483, N54533, N112765, N61339...","[N4858, N112472]","[[N118944, N27427, N78699, N126722, N35936, N2..."
...,...,...,...,...,...
298707,11/9/2019 9:59:56 AM,[N9935-0 N34971-0 N52101-0 N97499-0 N46106-0 N...,"[[N126878, N42094, N13604, N106155, N77079, N8...","[N83568, N63286, N3248, N86828, N15736, N31323...","[[N9935, N34971, N52101, N97499, N46106, N1063..."
298708,11/9/2019 9:59:57 AM,[N75842-0 N102389-0 N52435-0 N66489-0 N14551-0...,"[[N68431, N120756, N28820, N22756, N90561, N10...","[N62595, N76810]","[[N75842, N102389, N52435, N66489, N14551, N95..."
298709,11/9/2019 9:59:58 AM,[N4858-0 N66489-0 N63833-0 N64149-0 N66005-0 N...,"[[N82779, N61041, N104475, N61504, N128965, N6...","[N99362, N119637, N67206, N119637, N88891, N57...","[[N4858, N66489, N63833, N64149, N66005, N2793..."
298710,11/9/2019 9:59:58 PM,[N4858-0 N3604-0 N84137-0 N25037-0 N26404-0 N7...,"[[N83623, N16123, N37770], [N83623, N16123, N3...","[N70497, N81697, N97946, N93934]","[[N4858, N3604, N84137, N25037, N26404, N76810..."


In [23]:
# max_length = grouped_filtered_df_2_t['timestamp'].apply(len).max()
# mean_length = grouped_filtered_df_2_t['timestamp'].apply(len).mean()
# min_length = grouped_filtered_df_2_t['timestamp'].apply(len).min()
# mode_length = grouped_filtered_df_2_t['timestamp'].apply(len).mode().iloc[0]
# median_length = grouped_filtered_df_2_t['timestamp'].apply(len).median()

# # Print the results
# print("Max length:", max_length)
# print("Mean length:", mean_length)
# print("Min length:", min_length)
# print("Mode length:", mode_length)
# print("Median length:", median_length)

In [24]:
news = pd.read_csv(
    base_path/ Path("news.tsv"), 
    sep="\t",
    names=["itemId","category","subcategory","title","abstract","url","title_entities","abstract_entities"])
print(f"The article data consist in total of {len(news)} number of articles.")
news

The article data consist in total of 101527 number of articles.


,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
...,...,...,...,...,...,...,...,...
101522,N115249,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]
101523,N64337,finance,finance-real-estate,Mansion Monday: Contemporary Des Moines home i...,Among the perks of this unique Des Moines home...,https://assets.msn.com/labs/mind/BBWzQq8.html,"[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W...","[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W..."
101524,N100102,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[]
101525,N74617,autos,autossports,Best Sports Car Deals for October,NaN,https://assets.msn.com/labs/mind/BBy5rVe.html,"[{""Label"": ""Peugeot RCZ"", ""Type"": ""V"", ""Wikida...",[]


In [25]:
categories = news['category'].unique()

In [26]:
# category_num_mapping = {category: num for num, category in enumerate(news['category'].unique())}

# # Create a new column 'num' in the news DataFrame based on the mapping
# news['num_category'] = news['category'].map(category_num_mapping)

In [27]:
# category_num_mapping

In [28]:
# for r in filtered_df_2_1[0]:
#     print(r)

In [29]:
# news['num_category'] = news['category'].factorize()[0]
# article_category_map = news.set_index('itemId')['num_category'].to_dict()
# # def get_category_list(row):
# #     if pd.notna(row['click_history']):
# #         # Use a list comprehension to create the list of categories
# #         categories=[0] * 18    
# #         for article_id in row['click_history'].split():
            
# #             i=article_category_map.get(article_id, -1)
# #             if i!=-1:
# #                 categories[i]+=1
# #     else:
# #         # If 'click_history' is NaN, assign an empty list
# #         categories = []
# #     return categories
# def get_category_list(row):
#     if pd.notna(row['click_history']):
#         # Use a list comprehension to create the list of categories
#         categories = [article_category_map.get(article_id, 0) for article_id in row['click_history'].split()]
#         count_categories = [categories.count(i) for i in range(0, 18)]
#     else:
#         # If 'click_history' is NaN, assign a list of zeros
#         count_categories = []
#     return count_categories

# # Apply the function to create the new column
# filtered_df_2_1['category_list'] = filtered_df_2_1.apply(get_category_list, axis=1)


In [30]:
filtered_df_2_1

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N94157,"[N78206, N26368, N7578, N58592, N19858, N58258..."
1,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N78699,"[N78206, N26368, N7578, N58592, N19858, N58258..."
2,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N71090,"[N78206, N26368, N7578, N58592, N19858, N58258..."
3,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N31174,"[N78206, N26368, N7578, N58592, N19858, N58258..."
4,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,N25587,"[N47996, N82719, N117066, N8491, N123784, N214..."
...,...,...,...,...,...,...,...
1763133,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N39887,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763134,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N110709,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763135,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N24001,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763136,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N127572,"[N39887, N22811, N110709, N1923, N24001, N7667..."


In [31]:
# article_category_map.get('N45706', 0)

In [32]:
# feather_path=base_path/ Path("MINDlarge_train/category.feather")
# filtered_df_2_1.to_feather(feather_path)

In [33]:
# print(article_category_map.get('N88753', 0))
# article_category_map
# for article_id in filtered_df_2_1['click_history']:
#     print(article_id)
#     print(type(article_id))
#     print(article_category_map.get(article_id, 0))
#     break

In [34]:
filtered_df_2

,index,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,"[N94157, N78699, N71090, N31174]","[N78206, N26368, N7578, N58592, N19858, N58258..."
1,1,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,"[N25587, N36266]","[N47996, N82719, N117066, N8491, N123784, N214..."
2,13,14,U95671,11/14/2019 8:28:43 PM,"[N64593, N82779, N33216, N9321, N128643, N6449...",N62736-0 N52388-0 N16161-0 N127585-0 N88596-0 ...,"[N51048, N39403]","[N62736, N52388, N16161, N127585, N88596, N433..."
3,18,19,U419371,11/12/2019 4:15:12 AM,"[N129790, N59633, N51088, N126521, N882, N4570...",N113644-0 N22659-0 N59301-0 N78239-0 N91047-0 ...,"[N120192, N87838, N95254, N53859]","[N113644, N22659, N59301, N78239, N91047, N416..."
4,24,25,U552512,11/12/2019 8:51:10 PM,"[N114076, N83597, N38169, N8402, N3773, N10604...",N5098-1 N82866-0 N58417-0 N74117-0 N33468-0 N2...,"[N5098, N75494, N47643, N80444, N44037, N18947...","[N5098, N82866, N58417, N74117, N33468, N26837..."
...,...,...,...,...,...,...,...,...
615591,2232732,2232733,U510519,11/12/2019 6:24:48 AM,"[N70946, N57593, N9276, N5163, N63379, N40628,...",N43085-0 N100073-1 N20990-0 N2750-0 N27097-0 N...,"[N100073, N30899, N83412]","[N43085, N100073, N20990, N2750, N27097, N1217..."
615592,2232735,2232736,U182931,11/14/2019 6:03:16 AM,"[N87446, N104938, N49247, N112324, N103181, N8...",N18405-0 N76665-0 N25751-0 N5747-0 N85664-0 N4...,"[N20250, N58641]","[N18405, N76665, N25751, N5747, N85664, N45522..."
615593,2232737,2232738,U424578,11/13/2019 10:44:02 AM,"[N65123, N63858, N2661, N40628, N98238, N98238...",N88472-0 N106659-0 N51686-0 N10285-0 N127836-0...,"[N100417, N72609, N93856]","[N88472, N106659, N51686, N10285, N127836, N48..."
615594,2232740,2232741,U189711,11/14/2019 8:17:31 PM,"[N26834, N86208, N83597, N90359, N91280, N5531...",N72848-0 N104990-0 N3623-0 N51307-0 N83707-0 N...,"[N91238, N86258, N102846]","[N72848, N104990, N3623, N51307, N83707, N1186..."


In [35]:
import torch
def sampled_user(count_list):
    probabilities_tensor = torch.tensor(count_list, dtype=torch.float32)
    probabilities_normalized = probabilities_tensor / probabilities_tensor.sum()

    # Sample a value based on the probabilities
    sampled_value = torch.multinomial(probabilities_normalized, 1).item()

    # Create the sampled tensor
    sampled_list_tensor = torch.zeros_like(probabilities_tensor, dtype=torch.float32)
    sampled_list_tensor[sampled_value] = 1
    return sampled_list_tensor


/home/aayush/rl_mind_dataset/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
# sampled_user(filtered_df_2_1['category_list'].loc[0])

In [37]:
# user_state=torch.Tensor(filtered_df_2_1['category_list'].loc[0])
# user_state.shape
# print(user_state)


In [38]:
items=filtered_df_2_1['presented_slate'].loc[0]
items

['N78206',
 'N26368',
 'N7578',
 'N58592',
 'N19858',
 'N58258',
 'N18478',
 'N2591',
 'N97778',
 'N32954',
 'N94157',
 'N39404',
 'N108809',
 'N78699',
 'N71090',
 'N40282',
 'N31174',
 'N37924',
 'N27822']

In [39]:
# import random
# item_list = [article_category_map.get(key, []) for key in items]

# remaining_items = 30 - len(item_list)
# additional_values = random.choices(list(set(range(18)) - set(item_list)), k=remaining_items)

# item_list.extend(additional_values)

# random.shuffle(item_list)


In [40]:
# set(range(18))

In [41]:
# length_of_list = len(item_list)

# # Create a list of tensors
# tensor_list = [torch.eye(19)[value] for value in item_list]

# # Print the result
# for i, tensor in enumerate(tensor_list):
#     print(f"Tensor for value {item_list[i]}:\n{tensor}")

In [42]:
user_id = filtered_df_2_1['userId'].loc[5]
timestamp = filtered_df_2_1['timestamp'].loc[5]
clicked_items = filtered_df_2_1[(filtered_df_2_1['userId'] == user_id) & (filtered_df_2_1['timestamp'] == timestamp)]['click'].tolist()
print(clicked_items)



['N25587', 'N36266']


In [43]:
# import numpy as np
# numpy_array = np.copy(
#            filtered_df_2_1["category_list"].loc[20176]
#         )
# user_state = torch.Tensor(numpy_array)
# print(user_state)

In [44]:
filtered_df_2_1.loc[35683]

impressionId                                                   44943
userId                                                       U363502
timestamp                                     11/11/2019 10:27:41 AM
click_history      [N12194, N58990, N33543, N37215, N115522, N111...
impressions        N98178-1 N28752-0 N125945-0 N12613-0 N114526-0...
click                                                         N98178
presented_slate    [N98178, N28752, N125945, N12613, N114526, N47...
Name: 35683, dtype: object

In [45]:
news_df = pd.read_feather(
            base_path / Path("MINDlarge_train/news_glove_embed_50.feather")
        )
embedding_dict = dict(zip(news_df["itemId"], news_df["embedding"]))


In [46]:
filtered_df_2_1

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N94157,"[N78206, N26368, N7578, N58592, N19858, N58258..."
1,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N78699,"[N78206, N26368, N7578, N58592, N19858, N58258..."
2,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N71090,"[N78206, N26368, N7578, N58592, N19858, N58258..."
3,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N31174,"[N78206, N26368, N7578, N58592, N19858, N58258..."
4,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,N25587,"[N47996, N82719, N117066, N8491, N123784, N214..."
...,...,...,...,...,...,...,...
1763133,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N39887,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763134,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N110709,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763135,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N24001,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1763136,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N127572,"[N39887, N22811, N110709, N1923, N24001, N7667..."


In [47]:
def user_embedding(row):
  
    item_embeddings = [embedding_dict[item_id] for item_id in row['click_history'] if item_id in embedding_dict]
    item_embeddings = [embedding for embedding in item_embeddings if embedding is not None]
    if item_embeddings:
        user_embedding = sum(item_embeddings) / len(item_embeddings)
    else:

        user_embedding = None
    return user_embedding

# Apply the function to create the new column
filtered_df_2_1['observed_state'] = filtered_df_2_1.apply(user_embedding, axis=1)

In [48]:
filtered_df_2_1

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate,observed_state
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N94157,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
1,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N78699,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
2,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N71090,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
3,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N31174,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
4,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,N25587,"[N47996, N82719, N117066, N8491, N123784, N214...","[0.19647501, 0.16715528, -0.047917686, 0.01953..."
...,...,...,...,...,...,...,...,...
1763133,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N39887,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763134,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N110709,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763135,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N24001,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763136,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N127572,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."


In [49]:
has_empty_list = any(filtered_df_2_1['presented_slate'].apply(lambda x: len(x) < 2))

if has_empty_list:
    print("The 'presented_slate' column contains at least one empty list.")
else:
    print("No empty lists found in the 'presented_slate' column.")

No empty lists found in the 'presented_slate' column.


In [50]:
# feather_path=base_path/ Path("MINDlarge_train/interaction_all_50.feather")
# filtered_df_2_1.to_feather(feather_path)

In [51]:
import numpy as np
num_rows = len(filtered_df_2_1)
random_index = np.random.randint(0, num_rows)
items = filtered_df_2_1["presented_slate"].loc[random_index]

item_list = [embedding_dict.get(key, []) for key in items]

In [52]:
len(item_list)

130

In [53]:
k=100
all_vectors = [np.array(vector) for vector in embedding_dict.values()]

    # Convert item_list vectors to NumPy arrays
item_list_arrays = [np.array(vector) for vector in item_list]

# Filter out vectors that are already in item_list
available_vectors = [vector for vector in all_vectors if not any(np.array_equal(vector, item) for item in item_list_arrays)]

# If there are less than k available vectors, you can decide how to handle this situation
if len(available_vectors) < k:
    raise ValueError("Not enough available vectors to select k random vectors.")


In [54]:
filtered_df_2_1

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate,observed_state
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N94157,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
1,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N78699,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
2,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N71090,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
3,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N31174,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
4,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,N25587,"[N47996, N82719, N117066, N8491, N123784, N214...","[0.19647501, 0.16715528, -0.047917686, 0.01953..."
...,...,...,...,...,...,...,...,...
1763133,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N39887,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763134,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N110709,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763135,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N24001,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763136,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N127572,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."


In [55]:
column_name = 'click_history'
empty_values_mask = filtered_df_2_1[column_name].apply(lambda x: isinstance(x, list) and len(x) == 0)


# Display rows with empty or None values in the specified column
rows_with_empty_values = filtered_df_2_1[empty_values_mask]
print(rows_with_empty_values)

         impressionId   userId               timestamp click_history  \
83                121  U620890   11/13/2019 3:25:30 PM            []   
84                121  U620890   11/13/2019 3:25:30 PM            []   
187               272  U680579   11/13/2019 5:00:06 AM            []   
188               272  U680579   11/13/2019 5:00:06 AM            []   
404               483    U8626  11/14/2019 12:22:00 PM            []   
...               ...      ...                     ...           ...   
1763076       2232655  U184094   11/12/2019 7:05:55 AM            []   
1763077       2232655  U184094   11/12/2019 7:05:55 AM            []   
1763094       2232689  U548928   11/13/2019 7:12:53 PM            []   
1763095       2232689  U548928   11/13/2019 7:12:53 PM            []   
1763096       2232689  U548928   11/13/2019 7:12:53 PM            []   

                                               impressions    click  \
83       N123968-0 N39403-0 N10285-0 N16244-0 N94594-0 ...    N5

In [56]:
embedding_dict_1 = {key: value for key, value in embedding_dict.items() if value is not None}
has_none_values = any(value is None for value in embedding_dict_1.values())
print("Dictionary has None values:", has_none_values)

Dictionary has None values: False


In [57]:
valid_article_ids = set(embedding_dict_1.keys())

def filter_click_list(click_list):
    return [item for item in click_list if item in valid_article_ids]




filtered_df_2_1['presented_slate'] = filtered_df_2_1['presented_slate'].apply(filter_click_list)
filtered_df_2_1['click_history'] = filtered_df_2_1['click_history'].apply(filter_click_list)
# filtered_df_2_1[filtered_df_2_1['click'].apply(lambda x: any(item in valid_article_ids for item in x))]
# filtered_filtered_df_2_1 = filtered_filtered_df_2_1[filtered_filtered_df_2_1['presented_slate'].apply(lambda x: any(item in valid_article_ids for item in x))]
# filtered_filtered_df_2_1 = filtered_df_2_1[filtered_df_2_1['click'].apply(lambda x: len(x) > 0)]
filtered_df_2_1

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate,observed_state
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N94157,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
1,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N78699,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
2,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N71090,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
3,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N31174,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
4,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,N25587,"[N47996, N82719, N117066, N8491, N123784, N214...","[0.19647501, 0.16715528, -0.047917686, 0.01953..."
...,...,...,...,...,...,...,...,...
1763133,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N39887,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763134,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N110709,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763135,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N24001,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763136,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N127572,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."


In [58]:
filtered_df_2_1['click'].dtype

dtype('O')

In [59]:
filtered_df_2_2 = filtered_df_2_1[filtered_df_2_1['click'].isin(valid_article_ids)]

In [60]:
filtered_df_2_2

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate,observed_state
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N94157,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
1,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N78699,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
2,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N71090,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
3,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N31174,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
4,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,N25587,"[N47996, N82719, N117066, N8491, N123784, N214...","[0.19647501, 0.16715528, -0.047917686, 0.01953..."
...,...,...,...,...,...,...,...,...
1763133,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N39887,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763134,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N110709,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763135,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N24001,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763136,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N127572,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."


In [61]:
# feather_path=base_path/ Path("MINDlarge_train/interaction_all_50.feather")
# filtered_df_2_2.to_feather(feather_path)

In [62]:
# filtered_df_2_2['observed_state'].loc[1171843]

In [63]:
filtered_df_2_2

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate,observed_state
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N94157,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
1,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N78699,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
2,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N71090,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
3,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N31174,"[N78206, N26368, N7578, N58592, N19858, N58258...","[0.22171037, 0.20275427, -0.0017405786, -0.102..."
4,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,N25587,"[N47996, N82719, N117066, N8491, N123784, N214...","[0.19647501, 0.16715528, -0.047917686, 0.01953..."
...,...,...,...,...,...,...,...,...
1763133,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N39887,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763134,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N110709,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763135,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N24001,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."
1763136,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N127572,"[N39887, N22811, N110709, N1923, N24001, N7667...","[0.264194, 0.35523272, -0.24210165, 0.29260483..."


In [64]:
filtered_df_2_2.reset_index(inplace=True, drop=True)

In [65]:
filtered_df_2_2['observed_state'].loc[1171843]

array([ 0.23155305,  0.09046899, -0.04316003, -0.02428568,  0.31497824,
        0.20126353, -0.34392884, -0.01460247, -0.1693953 , -0.11472072,
       -0.14011937, -0.02910527, -0.3866399 , -0.10174464,  0.38473865,
        0.05010825, -0.0571515 , -0.1587927 , -0.26580936, -0.24946436,
        0.11636929,  0.22094035,  0.15056613, -0.140398  ,  0.02470146,
       -1.6998686 , -0.21147606,  0.01224888,  0.02680144, -0.1194284 ,
        2.95827   ,  0.04026161, -0.38374895, -0.38526583, -0.03111691,
       -0.11767005,  0.04738865,  0.02049944,  0.03631974, -0.15501308,
       -0.18096337,  0.19296144, -0.02373012,  0.05143049, -0.03698731,
        0.05729293, -0.23752818,  0.00867238,  0.01736   , -0.08155228],
      dtype=float32)

In [66]:
# feather_path=base_path/ Path("MINDlarge_train/interaction_all_50.feather")
# filtered_df_2_2.to_feather(feather_path)

In [67]:
training_data=filtered_df_2_2.loc[:, ["userId", "click_history"]]

In [68]:
embedding_dict_1

{'N88753': array([ 0.13696434,  0.13584633, -0.03737339,  0.01329388,  0.32152557,
         0.16187051, -0.5727945 , -0.28476918, -0.33877823, -0.12141334,
         0.0768334 ,  0.11145857, -0.30549335, -0.1405605 ,  0.38148436,
         0.16667122, -0.08279032, -0.034354  , -0.3252018 , -0.4397647 ,
        -0.06955055,  0.22050339,  0.346705  , -0.06001468, -0.02887418,
        -1.3902683 , -0.36114776,  0.05007345,  0.25905198, -0.54441345,
         3.1919403 ,  0.2859994 , -0.1482479 , -0.2124811 ,  0.14560944,
         0.03914554,  0.03940056,  0.16604204,  0.18811005,  0.02301063,
        -0.09415931,  0.19112554,  0.15002733,  0.10092195,  0.07866517,
         0.18207178, -0.21037501, -0.25587493, -0.19432306,  0.03306768],
       dtype=float32),
 'N45436': array([ 0.22841051,  0.24642792,  0.10140069,  0.21330339,  0.0853757 ,
         0.12618123, -0.82540804, -0.39293486, -0.03661034,  0.15733679,
        -0.12166972, -0.04666116, -0.5692446 ,  0.06465247,  0.67014945,
       

In [69]:
training_data = training_data.drop_duplicates(subset=["userId"])
training_data = training_data.reset_index(drop=True)
training_data

,userId,click_history
0,U87243,"[N8668, N39081, N65259, N79529, N73408, N43615..."
1,U598644,"[N56056, N8726, N70353, N67998, N83823, N11110..."
2,U95671,"[N64593, N82779, N33216, N9321, N128643, N6449..."
3,U419371,"[N129790, N59633, N51088, N126521, N882, N4570..."
4,U552512,"[N114076, N83597, N38169, N8402, N3773, N10604..."
...,...,...
323423,U248404,"[N97313, N90949, N104904, N17424, N18787, N129..."
323424,U636410,"[N128643, N14696, N26834, N31043, N67397, N725..."
323425,U182931,"[N87446, N104938, N49247, N112324, N103181, N8..."
323426,U189711,"[N26834, N86208, N83597, N90359, N55318, N8595..."


In [70]:
# import random

# all_items=set(embedding_dict_1.keys())
# # Reset the index

# def get_negative_items(click_history):
#     click_set = set(click_history)
#     return np.random.choice(list(all_items - click_set), size=10, replace=False)

# # Apply the function to the click_history column
# training_data['negative_items'] = training_data['click_history'].apply(get_negative_items)



# Drop the original click_history column
# training_data = training_data.drop(columns=["click_history"])

In [71]:
training_data

,userId,click_history
0,U87243,"[N8668, N39081, N65259, N79529, N73408, N43615..."
1,U598644,"[N56056, N8726, N70353, N67998, N83823, N11110..."
2,U95671,"[N64593, N82779, N33216, N9321, N128643, N6449..."
3,U419371,"[N129790, N59633, N51088, N126521, N882, N4570..."
4,U552512,"[N114076, N83597, N38169, N8402, N3773, N10604..."
...,...,...
323423,U248404,"[N97313, N90949, N104904, N17424, N18787, N129..."
323424,U636410,"[N128643, N14696, N26834, N31043, N67397, N725..."
323425,U182931,"[N87446, N104938, N49247, N112324, N103181, N8..."
323426,U189711,"[N26834, N86208, N83597, N90359, N55318, N8595..."


In [72]:
def split_click_history(click_history):
    if len(click_history) > 0:
        user_embedding = click_history[0]
        remaining_clicks = click_history[1:]
        return user_embedding, remaining_clicks
    else:
        return None, []

# Apply the function to the click_history column
training_data[["user_embedding", "click_history"]] = training_data["click_history"].apply(split_click_history).apply(pd.Series)

In [73]:
training_data

,userId,click_history,user_embedding
0,U87243,"[N39081, N65259, N79529, N73408, N43615, N2937...",N8668
1,U598644,"[N8726, N70353, N67998, N83823, N111108, N1075...",N56056
2,U95671,"[N82779, N33216, N9321, N128643, N64496, N6809...",N64593
3,U419371,"[N59633, N51088, N126521, N882, N45706, N48876...",N129790
4,U552512,"[N83597, N38169, N8402, N3773, N106046, N59981...",N114076
...,...,...,...
323423,U248404,"[N90949, N104904, N17424, N18787, N129836, N50...",N97313
323424,U636410,"[N14696, N26834, N31043, N67397, N72571, N1424...",N128643
323425,U182931,"[N104938, N49247, N112324, N103181, N84940, N1...",N87446
323426,U189711,"[N86208, N83597, N90359, N55318, N85952, N6240...",N26834


In [74]:
training_data=training_data.explode('click_history', ignore_index=True)


In [75]:
training_data

,userId,click_history,user_embedding
0,U87243,N39081,N8668
1,U87243,N65259,N8668
2,U87243,N79529,N8668
3,U87243,N73408,N8668
4,U87243,N43615,N8668
...,...,...,...
8482672,U189711,N92073,N26834
8482673,U189711,N120854,N26834
8482674,U189711,N110982,N26834
8482675,U189711,N110550,N26834


In [76]:
training_data['click'] = 1

In [96]:
training_sample=training_data.sample(n=60000, random_state=1)

In [97]:
embedding_dict_1

{'N88753': array([ 0.13696434,  0.13584633, -0.03737339,  0.01329388,  0.32152557,
         0.16187051, -0.5727945 , -0.28476918, -0.33877823, -0.12141334,
         0.0768334 ,  0.11145857, -0.30549335, -0.1405605 ,  0.38148436,
         0.16667122, -0.08279032, -0.034354  , -0.3252018 , -0.4397647 ,
        -0.06955055,  0.22050339,  0.346705  , -0.06001468, -0.02887418,
        -1.3902683 , -0.36114776,  0.05007345,  0.25905198, -0.54441345,
         3.1919403 ,  0.2859994 , -0.1482479 , -0.2124811 ,  0.14560944,
         0.03914554,  0.03940056,  0.16604204,  0.18811005,  0.02301063,
        -0.09415931,  0.19112554,  0.15002733,  0.10092195,  0.07866517,
         0.18207178, -0.21037501, -0.25587493, -0.19432306,  0.03306768],
       dtype=float32),
 'N45436': array([ 0.22841051,  0.24642792,  0.10140069,  0.21330339,  0.0853757 ,
         0.12618123, -0.82540804, -0.39293486, -0.03661034,  0.15733679,
        -0.12166972, -0.04666116, -0.5692446 ,  0.06465247,  0.67014945,
       

In [98]:
num_users = len(set(u for u in training_sample['userId']))
num_items = len(set(i for i in training_sample['click_history']))

In [99]:
training_sample

,userId,click_history,user_embedding,click
7765862,U660985,N48215,N65772,1
562617,U381838,N102132,N12959,1
3484051,U423221,N56178,N110755,1
504605,U181626,N3948,N2484,1
1165139,U368726,N1014,N109183,1
...,...,...,...,...
2772826,U573131,N86409,N123702,1
4643487,U464393,N37721,N9740,1
7783058,U415825,N91597,N122359,1
8378893,U105287,N51657,N110755,1


In [100]:
interactions=training_sample[['user_embedding','click_history']].reset_index(drop=True)
interactions['click'] = 1

In [101]:
interactions

,user_embedding,click_history,click
0,N65772,N48215,1
1,N12959,N102132,1
2,N110755,N56178,1
3,N2484,N3948,1
4,N109183,N1014,1
...,...,...,...
59995,N123702,N86409,1
59996,N9740,N37721,1
59997,N122359,N91597,1
59998,N110755,N51657,1


In [102]:
print(interactions['user_embedding'].nunique())

6217


In [103]:
import random
all_items=set(embedding_dict_1.keys())
additional_item_ids = all_items - set(interactions.loc[interactions['click'] == 1, 'click_history'])
additional_rows = []

# For each unique user ID, randomly select k item IDs from additional item IDs
k = 10  # Example value of k
for user_id in interactions['user_embedding'].unique():
    additional_item_ids_for_user = random.sample(list(additional_item_ids), k)
    for additional_item_id in additional_item_ids_for_user:
        additional_rows.append({'user_embedding': user_id, 'click_history': additional_item_id, 'click': 0})

# Concatenate original DataFrame and additional DataFrame
additional_df = pd.DataFrame(additional_rows)
interactions1 = pd.concat([interactions, additional_df], ignore_index=True)


In [104]:
interactions2 = interactions1.sample(frac=1).reset_index(drop=True)
interactions2
# user_item_interactions = list(interactions.itertuples(index=False, name=None))

,user_embedding,click_history,click
0,N78695,N3781,0
1,N94734,N124053,1
2,N21650,N111249,0
3,N80686,N48095,0
4,N53897,N91809,0
...,...,...,...
122175,N62780,N23574,0
122176,N5726,N125543,0
122177,N91,N48475,1
122178,N74092,N91810,1


In [85]:
# user_item_interactions

In [86]:
# positive_interactions = {}
# for user, item in user_item_interactions:
#     if user not in positive_interactions:
#         positive_interactions[user] = []
#     positive_interactions[user].append(item)

In [87]:
all_items=set(embedding_dict_1.keys())

In [88]:
# user_item_pairs = []
# for user, pos_items in positive_interactions.items():
#     for pos_item in pos_items:
#         # Positive interaction
#         # user_item_pairs.append((user, pos_item))
        
#         # Negative interactions (randomly sampled)
#         neg_items = all_items - set(pos_items)  # All items not interacted with
#         neg_item = np.random.choice(list(neg_items))  # Randomly select a negative item
#         user_item_pairs.append((user, pos_item, neg_item))

In [89]:
# user_item_pairs

In [90]:
# user_item_pairs_list = []
# embedding_size=50
# default_embedding = np.zeros(embedding_size)
# for triple in user_item_pairs:
#     triple_with_values = tuple(embedding_dict_1.get(item_id,default_embedding) for item_id in triple)
#     user_item_pairs_list.append(triple_with_values)

In [91]:
# user_item_pairs_list

In [92]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import Dataset

# Define the Neural Collaborative Filtering (NCF) model
class NCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=64, hidden_dim=64):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.fc_layers = nn.Sequential(
            nn.Linear(2 * embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, user_ids, item_ids,train=True):
        # print(user_ids)
        user_embeds = user_ids #torch.tensor([embedding_dict_1[item_id] for item_id in user_ids])
        item_embeds = item_ids#torch.tensor([embedding_dict_1[item_id] for item_id in item_ids])  # Retrieve item embeddings
        if train:
            concat_embeds = torch.cat([user_embeds, item_embeds], dim=1)
        else:
            concat_embeds = torch.cat([user_embeds, item_embeds], dim=0)
        output = self.fc_layers(concat_embeds)
        return output.squeeze()

# Preprocess data
# user_encoder = LabelEncoder()
# item_encoder = LabelEncoder()
# training_sample['user_embedding'] = user_encoder.fit_transform(training_sample['user_embedding'])
# training_sample['item_id'] = item_encoder.fit_transform(training_sample['click_history'])
num_users = len(set(interactions['user_embedding']))
num_items = len(set(interactions['click_history']))

# Split data into train and test sets
# train_df, test_df = train_test_split(interactions2, test_size=0.2, random_state=42)


class DataFrameDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        # print(row['user_embedding'])
        default_embedding = torch.zeros(50, dtype=torch.float32)
        user_id = torch.tensor(embedding_dict_1.get(row['user_embedding'],default_embedding))
        item_id = torch.tensor(embedding_dict_1.get(row['click_history'],default_embedding))
        rating = torch.tensor(row['click']).to(torch.float32)
        return {'user_id': user_id, 'item_id': item_id, 'rating': rating}
    
dataset=DataFrameDataset(interactions2)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)




# Instantiate the NCF model
model = NCF(num_users=num_users, num_items=num_items, embedding_dim=50, hidden_dim=50)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
# for batch in train_loader:
#     print(batch)
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
      
        user_ids = batch['user_id']
        
        item_ids = batch['item_id']
        ratings = batch['rating']
        
        optimizer.zero_grad()
        outputs = model(user_ids, item_ids)
        # print(outputs.dtype)
        # print(ratings.dtype)
        loss = criterion(outputs, ratings)
        loss.backward()
        optimizer.step()
    print(f"epoch: {epoch}, loss: {loss}")

# Example of making predictions
model.eval()
with torch.no_grad():
    
    user_embedding = torch.tensor(embedding_dict_1.get(interactions.loc[656,'user_embedding'])) # Example user ID
    item_id =torch.tensor(embedding_dict_1.get(interactions.loc[659,'click_history']))  # Example item ID
    rating_prediction = torch.sigmoid(model(user_embedding, item_id,train=False))
    print(rating_prediction)



/tmp/ipykernel_1085380/3696304394.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  user_id = torch.tensor(embedding_dict_1.get(row['user_embedding'],default_embedding))
/tmp/ipykernel_1085380/3696304394.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item_id = torch.tensor(embedding_dict_1.get(row['click_history'],default_embedding))


epoch: 0, loss: 0.29083141684532166
tensor(0.8824)


In [106]:
interactions2

,user_embedding,click_history,click
0,N78695,N3781,0
1,N94734,N124053,1
2,N21650,N111249,0
3,N80686,N48095,0
4,N53897,N91809,0
...,...,...,...
122175,N62780,N23574,0
122176,N5726,N125543,0
122177,N91,N48475,1
122178,N74092,N91810,1


In [105]:
num_ones = (interactions2['click'] == 1).sum()

print("Number of 1's in the 'click' column:", num_ones)

Number of 1's in the 'click' column: 60000


In [108]:
feather_path=base_path/ Path("MINDlarge_train/choice_model_data.feather")
interactions2.to_feather(feather_path)